In [10]:
from PIL import Image
from direct.preprocessor import Preprocessor, pose_inv, SceneData
import numpy as np
import open3d as o3d
import json

dir = "experiments/wood"

demo_head_rgb = np.array(Image.open("../{0}/demo_head_rgb.png".format(dir)))
demo_head_depth = np.array(Image.open("../{0}/demo_head_depth.png".format(dir)))
demo_head_mask = np.array(Image.open("../{0}/demo_head_seg.png".format(dir)))

demo_wrist_rgb = np.array(Image.open("../{0}/demo_wrist_rgb.png".format(dir)))
demo_wrist_depth = np.array(Image.open("../{0}/demo_wrist_depth.png".format(dir)))
demo_wrist_mask = np.array(Image.open("../{0}/demo_wrist_seg.png".format(dir)))

intrinsics_d415 = np.load("../handeye/intrinsics_d415.npy")
intrinsics_d405 = np.load("../handeye/intrinsics_d405.npy")

data = SceneData(
    image_0=demo_head_rgb,
    image_1=demo_wrist_rgb,
    depth_0=demo_head_depth,
    depth_1=demo_wrist_depth,
    seg_0=demo_head_mask,
    seg_1=demo_wrist_mask,
    intrinsics_0=intrinsics_d415,
    intrinsics_1=intrinsics_d405,
    T_WC=np.eye(4) # cam frame
)

processor = Preprocessor()
data.update(processor(data))

In [11]:
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(data["pc1"][:, :3])
o3d.visualization.draw_geometries([pcd1])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [12]:
import copy

points_wrist_cam = copy.deepcopy(data["pc1"][:, :3])
ones = np.ones((points_wrist_cam.shape[0], 1))

# Append the column of ones to the original points array
points_wrist_cam_with_ones = np.hstack((points_wrist_cam, ones))

points_wrist_cam_with_ones.shape

(42964, 4)

In [13]:
T_C_EEF = np.load("../handeye/T_C_EEF_wrist_l.npy")

T_C_EEF @ np.array([1,1,1,1])

array([-1.02713561, -0.8344819 ,  1.08403704,  1.        ])

In [14]:
from scipy.spatial.transform import Rotation as R

def create_homogeneous_matrix(xyz, quaternion):
    rotation_matrix = R.from_quat(quaternion).as_matrix()
    T = np.eye(4)  # Start with an identity matrix
    T[:3, :3] = rotation_matrix  # Insert the rotation matrix
    T[:3, 3] = xyz  # Insert the translation vector

    return T

T_WC = np.load("../handeye/T_WC_head.npy")
T_C_EEF = np.load("../handeye/T_C_EEF_wrist_l.npy")
with open(f"../{dir}/demo_bottlenecks.json") as f:
    dbn = json.load(f)
T_EEF_WORLD = create_homogeneous_matrix(dbn["bottleneck_left"][:3], dbn["bottleneck_left"][3:])

T_wrist2head = pose_inv(T_WC) @ T_EEF_WORLD @ T_C_EEF

points_head_cam = []
for point in points_wrist_cam_with_ones:
    points_head_cam.append(T_wrist2head @ point)
points_head_cam = np.array(points_head_cam)

In [15]:
data["pc0"][:, :3]

array([[ 0.05467011, -0.0217233 ,  0.703     ],
       [ 0.0554427 , -0.0217233 ,  0.703     ],
       [ 0.05621529, -0.0217233 ,  0.703     ],
       ...,
       [ 0.09847307,  0.13067403,  0.66      ],
       [ 0.09934871,  0.13087203,  0.661     ],
       [ 0.10007515,  0.13087203,  0.661     ]], dtype=float32)

In [16]:
pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(data["pc0"][:, :3])
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(points_head_cam[:, :3])
o3d.visualization.draw_geometries([pcd0, pcd1])

# Calculate the centroid of each point cloud array([-0.00344304,  0.00669954,  0.00428747])
pcd0_centre = np.mean(data["pc0"][:, :3], axis=0)  # Calculate mean across columns (axis=0)
pcd1_centre = np.mean(points_head_cam[:, :3], axis=0)  # Calculate mean across columns (axis=0)

print(pcd0_centre)
print(pcd1_centre)
# # Compute the difference between the centroids
diff = pcd0_centre - pcd1_centre
diff

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[0.0371766  0.03764415 0.65259916]
[0.03624739 0.01021223 0.66775771]


array([ 0.00092921,  0.02743192, -0.01515856])

In [9]:
print('run Poisson surface reconstruction')
# Assuming data["pc0"] and data["pc1"] are your point cloud data
pcd0 = o3d.geometry.PointCloud()
pcd1 = o3d.geometry.PointCloud()

pcd0.points = o3d.utility.Vector3dVector(data["pc0"][:, :3])
pcd1.points = o3d.utility.Vector3dVector(points_head_cam[:, :3][:, :3])

# colored pointcloud registration
# This is implementation of following paper
# J. Park, Q.-Y. Zhou, V. Koltun,
# Colored Point Cloud Registration Revisited, ICCV 2017
voxel_radius = [0.04, 0.02, 0.01]
max_iter = [50, 30, 14]
current_transformation = np.identity(4)
print("3. Colored point cloud registration")
for scale in range(3):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter, radius, scale])

    print("3-1. Downsample with a voxel size %.2f" % radius)
    source_down = source.voxel_down_sample(radius)
    target_down = target.voxel_down_sample(radius)

    print("3-2. Estimate normal.")
    source_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
    target_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

    print("3-3. Applying colored point cloud registration")
    result_icp = o3d.pipelines.registration.registration_colored_icp(
        source_down, target_down, radius, current_transformation,
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=iter))
    current_transformation = result_icp.transformation
    print(result_icp)
    draw_registration_result_original_color(source, target,
                                            result_icp.transform)

run Poisson surface reconstruction
3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04


NameError: name 'source' is not defined

In [17]:
import open3d as o3d
import numpy as np
import copy
import cv2

# Assuming data["pc0"] and data["pc1"] are your point cloud data
pcd0 = o3d.geometry.PointCloud()
pcd1 = o3d.geometry.PointCloud()

pcd0.points = o3d.utility.Vector3dVector(data["pc0"][:, :3])
pcd1.points = o3d.utility.Vector3dVector(points_head_cam[:, :3])
pcd0.colors = o3d.utility.Vector3dVector(data["pc0"][:, 3:6])
pcd1.colors = o3d.utility.Vector3dVector(data["pc1"][:, 3:6])

# Estimate normals for each point cloud
pcd0.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=40))
pcd1.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=40))

# Function to draw registration results
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.transform(transformation)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, target_temp])

# Compute FPFH features
voxel_size = 0.05  # Set voxel size for downsampling (adjust based on your data)
source_down = pcd0.voxel_down_sample(voxel_size)
target_down = pcd1.voxel_down_sample(voxel_size)

source_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
target_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))

source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    source_down,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    target_down,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

# Global registration using RANSAC
distance_threshold = voxel_size * 1.5
result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, source_fpfh, target_fpfh, mutual_filter=False,
    max_correspondence_distance=distance_threshold,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(), 
    ransac_n=4,
    checkers=[
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9), 
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    ],
    criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
)

# Use the result of global registration as the initial transformation for ICP
trans_init = result.transformation

# Apply ICP
threshold = 0.01  # Set a threshold for ICP, this depends on your data
reg_p2p = o3d.pipelines.registration.registration_icp(
    pcd0, pcd1, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())

# Get the transformation matrix
T_delta_cam = reg_p2p.transformation

# Draw the result
draw_registration_result(pcd0, pcd1, T_delta_cam)

print(T_delta_cam)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[[ 0.97903383 -0.20356119 -0.00745706 -0.01175871]
 [ 0.1922788   0.93561638 -0.29605887  0.20526802]
 [ 0.06724304  0.28841781  0.95514059 -0.03505099]
 [ 0.          0.          0.          1.        ]]


: 

In [25]:


def translation_from_matrix(matrix):
    """Extracts the translation vector from a 4x4 homogeneous transformation matrix."""
    return matrix[:3, 3]

def quaternion_from_matrix(matrix):
    """Extracts the quaternion from a 4x4 homogeneous transformation matrix."""
    rotation_matrix = matrix[:3, :3]
    rotation = R.from_matrix(rotation_matrix)
    return rotation.as_quat()

translation_from_matrix(T_WC_new), quaternion_from_matrix(T_WC_new)

(array([ 1.02814375, -0.06392402,  0.59424258]),
 array([ 0.62529498,  0.60027432, -0.34746441, -0.35769457]))

In [112]:
# colored pointcloud registration
# This is implementation of following paper
# J. Park, Q.-Y. Zhou, V. Koltun,
# Colored Point Cloud Registration Revisited, ICCV 2017
voxel_radius = [0.04, 0.02, 0.01]
max_iter = [50, 30, 14]
current_transformation = np.identity(4)
print("3. Colored point cloud registration")
for scale in range(3):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter, radius, scale])

    print("3-1. Downsample with a voxel size %.2f" % radius)
    # source_down = source.voxel_down_sample(radius)
    # target_down = target.voxel_down_sample(radius)

    print("3-2. Estimate normal.")
    source_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
    target_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

    print("3-3. Applying colored point cloud registration")
    result_icp = o3d.pipelines.registration.registration_colored_icp(
        source_down, target_down, radius, current_transformation,
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=iter))
    current_transformation = result_icp.transformation
    print(result_icp)

# Get the transformation matrix
T_delta_cam = reg_p2p.transformation

# Draw the result
draw_registration_result(pcd0, pcd1, T_delta_cam)

3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration


TypeError: registration_colored_icp(): incompatible function arguments. The following argument types are supported:
    1. (source: open3d.cpu.pybind.geometry.PointCloud, target: open3d.cpu.pybind.geometry.PointCloud, max_correspondence_distance: float, init: numpy.ndarray[numpy.float64[4, 4]] = array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), estimation_method: open3d.cpu.pybind.pipelines.registration.TransformationEstimationForColoredICP = TransformationEstimationForColoredICP with lambda_geometric=0.968000, criteria: open3d.cpu.pybind.pipelines.registration.ICPConvergenceCriteria = ICPConvergenceCriteria class with relative_fitness=1.000000e-06, relative_rmse=1.000000e-06, and max_iteration=30) -> open3d.cpu.pybind.pipelines.registration.RegistrationResult

Invoked with: PointCloud with 16 points., PointCloud with 16 points., 0.04, array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

: 

In [13]:
T_WC = np.load("../handeye/T_WC_head.npy")
T_delta_world = T_WC @ T_delta_cam @ pose_inv(T_WC)
T_delta_world

array([[ 0.79857538, -0.60179267, -0.011088  ,  0.28067213],
       [ 0.60165335,  0.79864132, -0.01361215, -0.38310653],
       [ 0.01704703,  0.00419919,  0.99984587, -0.00643008],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [17]:
r = R.from_matrix(T_delta_world[:3, :3]).as_euler("xyz")
yaw_only_delta_rotation = R.from_euler("xyz", [0, 0, r[-1]]).as_matrix()
yaw_only_delta_rotation

array([[ 0.79869144, -0.60174079,  0.        ],
       [ 0.60174079,  0.79869144,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [10]:
from scipy.spatial.transform import Rotation as R


def translation_from_matrix(matrix):
    """Extracts the translation vector from a 4x4 homogeneous transformation matrix."""
    return matrix[:3, 3]

def euler_from_matrix(matrix):
    """Extracts the quaternion from a 4x4 homogeneous transformation matrix."""
    rotation_matrix = matrix[:3, :3].copy()
    rotation = R.from_matrix(rotation_matrix)
    return rotation.as_euler(seq="XYZ", degrees=True)

trans = translation_from_matrix(T_delta_world)
rotation = euler_from_matrix(T_delta_world)

print(trans, rotation)

[ 0.28067213 -0.38310653 -0.00643008] [ 0.77999087 -0.63530886 37.00105027]


In [17]:
def create_homogeneous_matrix(xyz, quaternion):
    # Convert the quaternion to a rotation matrix
    rotation_matrix = R.from_quat(quaternion).as_matrix()
    # Create a homogeneous transformation matrix
    T = np.eye(4)  # Start with an identity matrix
    T[:3, :3] = rotation_matrix  # Insert the rotation matrix
    T[:3, 3] = xyz  # Insert the translation vector

    return T

demo = [
    0.5041812568964179,
    0.09185436015924689,
    0.48143709451847916,
    -0.9983786626178943,
    0.013449729176136651,
    -0.054892707858185244,
    0.006778011389003352
]

T_eef = create_homogeneous_matrix(demo[:3], demo[3:])
T_eef

array([[ 0.99361179, -0.02611172,  0.10978974,  0.50418126],
       [-0.02759997, -0.99954633,  0.01205746,  0.09185436],
       [ 0.10942509, -0.01501063, -0.9938817 ,  0.48143709],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [35]:
T_delta_world @ T_eef

array([[ 0.80887007,  0.58083391,  0.09143946,  0.62268341],
       [ 0.57427787, -0.81378487,  0.08921382, -0.0129589 ],
       [ 0.12623046, -0.01965073, -0.99180629,  0.48391332],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [36]:
new_T @ T_eef

array([[ 0.74345614, -0.66244621,  0.09185851,  0.62268341],
       [-0.65977202, -0.74895908, -0.06132849, -0.0129589 ],
       [ 0.10942509, -0.01501063, -0.9938817 ,  0.48391332],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [56]:
T_bias = create_homogeneous_matrix([trans[0], trans[1], 0], R.from_euler('xyz', [0, 0, rotation[2]]).as_quat())
T_bias

array([[ 0.76608952,  0.64273389,  0.        ,  0.28067213],
       [-0.64273389,  0.76608952, -0.        , -0.38310653],
       [-0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [62]:
R.from_matrix(T_eef[:3, :3]).as_euler('xyz')

array([-3.12649077, -0.10964465, -0.02777028])

In [61]:
res = T_bias @ T_eef
R.from_matrix(res[:3, :3]).as_euler('xyz')


array([-3.12649077, -0.10964465, -0.72583185])

In [52]:
R.from_euler('xyz', [0, 0, rotation[2]]).as_matrix() @ T_eef[:3, :3]

array([[ 0.74345614, -0.66244621,  0.09185851],
       [-0.65977202, -0.74895908, -0.06132849],
       [ 0.10942509, -0.01501063, -0.9938817 ]])

In [37]:
def pose_inv(pose):
    """Inverse a 4x4 homogeneous transformation matrix."""
    R = pose[:3, :3]
    T = np.eye(4)
    T[:3, :3] = R.T
    T[:3, 3] = - R.T @ np.ascontiguousarray(pose[:3, 3])
    return T

In [44]:
import numpy as np
r_project = np.linalg.inv(R.from_euler('xyz', [rotation[0], rotation[1], rotation[2]]).as_matrix()) @ R.from_euler('xyz', [0, 0, rotation[2]]).as_matrix()
r_project

array([[ 8.04888450e-01, -6.66575509e-17,  5.93426139e-01],
       [-4.17340537e-01,  7.10919961e-01,  5.66056256e-01],
       [-4.21878488e-01, -7.03272926e-01,  5.72211266e-01]])

In [20]:
r = R.from_euler('xyz', [0, 0, rotation[-1]])

# Get the rotation matrix
adjusted_rotation_matrix = r.as_matrix()

adjusted_rotation_matrix

array([[ 0.76608952,  0.64273389,  0.        ],
       [-0.64273389,  0.76608952, -0.        ],
       [-0.        ,  0.        ,  1.        ]])

In [31]:
adjusted_translation = T_delta_world[:3, :3] @ T_eef[:3, 3] - adjusted_rotation_matrix @ T_eef[:3, 3] + T_delta_world[:3, 3]
adjusted_translation

array([0.17739752, 0.24072682, 0.00247622])

In [32]:
new_T =  create_homogeneous_matrix(adjusted_translation, r.as_quat())
new_T

array([[ 0.76608952,  0.64273389,  0.        ,  0.17739752],
       [-0.64273389,  0.76608952, -0.        ,  0.24072682],
       [-0.        ,  0.        ,  1.        ,  0.00247622],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [27]:
T_delta_world

array([[ 0.79857538, -0.60179267, -0.011088  ,  0.28067213],
       [ 0.60165335,  0.79864132, -0.01361215, -0.38310653],
       [ 0.01704703,  0.00419919,  0.99984587, -0.00643008],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [133]:
PointCloud = np.ndarray

def rotate_pointcloud(pcd: PointCloud, angle_z: float):
    print("predicted rotation", np.rad2deg(angle_z))
    R = np.eye(3)
    cosine = np.cos(angle_z)
    sine = np.sin(angle_z)
    R[0, 0] = cosine
    R[1, 1] = cosine
    R[0, 1] = -sine
    R[1, 0] = sine

    pcd[:3, :] = R @ pcd[:3, :]
    return R, pcd

def find_translation(pcd0: PointCloud, pcd1: PointCloud) -> np.ndarray:
    pcd0_centre = np.mean(pcd0[:3, :], axis=1)
    pcd1_centre = np.mean(pcd1[:3, :], axis=1)
    return pcd1_centre - pcd0_centre
    
R_mtx, rotated_pcd0 = rotate_pointcloud(data["pc0"], rotation[-1])
translation = find_translation(rotated_pcd0, data["pc1"])

print(translation)
T_delta_base = np.eye(4)
T_delta_base[:3, :3] = R_mtx
T_delta_base[:3, 3] = translation

T_delta_cam = pose_inv(data["T_WC"]) @ T_delta_base @ data["T_WC"]

print(T_delta_base)

predicted rotation -1854.694161068739
[-0.09844735 -0.39142355 -0.21084177]
[[ 0.57794079  0.8160787   0.         -0.09844735]
 [-0.8160787   0.57794079  0.         -0.39142355]
 [ 0.          0.          1.         -0.21084177]
 [ 0.          0.          0.          1.        ]]


In [63]:
import open3d as o3d
import numpy as np

width = 848  # Replace with your camera image width
height = 480 # Replace with your camera image height
fx = 431.56503296
fy = 431.18637085
cx = 418.71490479
cy = 235.15617371 
intrinsics = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

demo_mask = np.array(Image.open("../data/lego/demo_wrist_mask.png"))
demo_rgb = np.array(Image.open("../data/lego/demo_wrist_rgb.png")) 
demo_depth = np.array(Image.open("../data/lego/demo_wrist_depth.png")).astype(np.uint16)

color = o3d.geometry.Image(demo_rgb)
depth = o3d.geometry.Image(demo_depth)

rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color, depth, depth_scale=1000.0, convert_rgb_to_intensity=False)

rgbd_image
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsics) 

# Visualization
o3d.visualization.draw_geometries([pcd]) 